In [1]:
from config import DATABASE_URL
from src.utils.database_connection_utils import get_database_session

In [2]:
session, engine = get_database_session(DATABASE_URL)


In [3]:
sql_query = "select * from article"

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = None
with engine.connect() as connection:
    data = pd.read_sql_query(sql=sql_query, con=connection)

In [6]:
data.shape

(11218, 9)

In [7]:
data.website_origin.unique()

array(['https://forumdesas.net/', 'https://www.7sur7.cd',
       'https://www.politico.cd', 'https://www.actualite.cd',
       'https://www.mediacongo.net'], dtype=object)

In [8]:
data.head()

,id,title,content,sumary,posted_at,website_origin,url,author,saved_at
0,1,La Commission tarifaire propose des mesures d’...,Les membres de la Commission tarifaire viennen...,None,2022-09-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/la-commissi...,Kléber KUNGU,2022-05-12 16:53:56.464801+00:00
1,2,La Commission tarifaire pour la mise en oeuvre...,Les membres de la Commission tarifaire so...,None,2022-04-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/la-commissi...,Kléber KUNGU,2022-05-12 16:53:56.505433+00:00
2,3,Lancement du 1er Laboratoire de recherche et d...,Vodacom Congo vient de signer un partenariat a...,None,2022-04-23,https://forumdesas.net/,https://www.forumdesas.net/2022/04/lancement-d...,None,2022-05-12 16:53:56.548859+00:00
3,4,Léopards : Hectór Cúper ce mardi à Kinshasa,"Le sélectionneur des Léopards de la RDC, Hectó...",None,2022-03-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/leopards-he...,Gloire BATOMENE,2022-05-12 16:53:56.561548+00:00
4,5,» Air Congo » en voie d’être créée pour s’en...,Le protocole d’accord était déjà signé entre l...,None,2022-11-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/air-congo-e...,KANDOLO M.,2022-05-12 16:53:56.682724+00:00


In [44]:
random_index = np.random.randint(0, data.shape[0])
sample_doc = data.content[random_index]
sample_doc

'La Haute Cour Militaire a rejeté en bloc la requête sollicitant une enquête supplémentaire, introduite par la partie civile à l’audience d‘hier lundi 27 juillet 2015 à la Prison Centrale de Makala. Dans ce document lu séance tenante par la greffière, la partie civile exige la comparution de l’ancien Inspecteur général de la Police Nationale Congolaise, le général John Numbi, considéré par plusieurs ONG de défense des droits de l’homme comme le commanditaire de l’assassinat de Floribert Chebeya, ancien Directeur de la Voix des Sans Voix.La partie civile a réitéré également sa demande relative à la mise en place d’une Commission rogatoire chargée d’aller entendre le major Paul Milambwe à Dakar, où il se trouve à la disposition de la justice sénégalaise.Enfin, la partie civile a exigé, sur base de l’arrêt de la Haute Cour du mois d’octobre 2012, la descente d’une mission d’enquête dans la ferme de John Numbi pour vérifier si les restes du corps du chauffeur de Chebeya, feu Fidèle Bazana,

How should I split the text , should I use the sentence or the word level to detect names in this entities?

I will try sentence and see how the french NER model from spacy will perform and then I will try more complicated approaches.

By the end of this work we should have a pipeline that takes the text and mask the name entities.

In [45]:
from nltk.tokenize import sent_tokenize
from unicodedata import normalize

In [46]:
tokenized_document = sent_tokenize(normalize("NFKD", sample_doc), language="french")

In [47]:
tokenized_document

['La Haute Cour Militaire a rejeté en bloc la requête sollicitant une enquête supplémentaire, introduite par la partie civile à l’audience d‘hier lundi 27 juillet 2015 à la Prison Centrale de Makala.',
 'Dans ce document lu séance tenante par la greffière, la partie civile exige la comparution de l’ancien Inspecteur général de la Police Nationale Congolaise, le général John Numbi, considéré par plusieurs ONG de défense des droits de l’homme comme le commanditaire de l’assassinat de Floribert Chebeya, ancien Directeur de la Voix des Sans Voix.La partie civile a réitéré également sa demande relative à la mise en place d’une Commission rogatoire chargée d’aller entendre le major Paul Milambwe à Dakar, où il se trouve à la disposition de la justice sénégalaise.Enfin, la partie civile a exigé, sur base de l’arrêt de la Haute Cour du mois d’octobre 2012, la descente d’une mission d’enquête dans la ferme de John Numbi pour vérifier si les restes du corps du chauf

In [48]:

import spacy

In [49]:
nlp = spacy.load("fr_core_news_sm")

In [50]:
from spacy import displacy

In [51]:
processed_sentence = nlp(tokenized_document[1])
processed_document = nlp(sample_doc)

In [35]:
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [37]:
tokenized_document[0]

'Les jeunes entrepreneurs sélectionnés par le Projet d’appui au développement des micro, petites et moyennes entreprises (PADMPME), dans le cadre du concours des plans d’affaires (COPA), ont reçu l’information sur le processus et la procédure de décaissement de leur subvention, au cours d’une session d’information organisée à leur intention hier jeudi 13 janvier à Kinshasa.'

In [53]:
displacy.render(processed_sentence, style="ent", jupyter=True)

In [54]:
displacy.render(processed_document, style="ent", jupyter=True)

The spacy model seems to be do well in detecting names, organization  ,but it can be improve to detec orgainization and more place.

In [19]:
{"meta": 
    {"source": "BBC News","via":"News API"},
 "_task_hash":142354664,
 "text":"ViewRay (VRAY +1.2%) inks an agreement with an affiliate of Fosun International Limited for the direct sale of stock and warrants yielding ~$59.1M in gross proceeds.Specifically, the company will sell",
 "_input_hash":1622806810}

And improve the spacy annodation , if by saturaday I can'tafind anything I will need to continue with what I have and make it a baseline

In [59]:
def tokenize_document(document):
    """given a document split the document into sentences and return a list of those sentence

    Args:
        document (_type_): _description_

    Returns:
        _type_: _description_
    """
    cleaned_document = normalize("NFKD", document)
    tokenized_document = sent_tokenize(cleaned_document, language="french")
    return tokenized_document

In [60]:
def generate_sentences(document_serie):
    """takea serie of documents and return a list of sentences

    Args:
        document_serie (_type_): _description_

    Returns:
        _type_: _description_
    """
    for document in document_serie:
        sentences = tokenize_document(document)
        for sentence in sentences:
            yield sentence

In [71]:
from secrets import token_hex
import json

In [74]:
def build_json_document(document_series, output_file_path):
    """take a series of documents and return a list of json documents

    Args:
        document_series (_type_): _description_

    Returns:
        _type_: _description_
    """
    with open(output_file_path, "w") as output_file:
        for sentence in generate_sentences(document_series):
            item = {"_task_hash": token_hex(6), "_input_hash": token_hex(6),  "text": sentence}
            output_file.write(json.dumps(item, ensure_ascii=False) + "\n")

In [68]:
from pathlib import Path



In [75]:
corpus_path = Path.cwd().joinpath("data", "corpus", "congo_news.jsonl")
build_json_document(data.content, corpus_path)